# 3. Parameter Estimation for the Gamma Conjugate Prior
In this notebook, parameters for the gamma conjugate prior (GCP) are estimated using the
*least surprise estimate*, that is, by minimizing the maximum Kullback-Leibler distance
of the GCP to any of the regional distributions.


## Configuration
First, the configuration of the fitting algorithm. As described in the REHEATFUNQ model
description paper, the algorithm proceeds by randomly distributing disks across a spherical
Earth, and for each disk selecting data from the filtered global heat flow database that
are within the disk. The following parameters control the disk size and the disk acceptance
criterion:

| Parameter    | Purpose                                                                                       |
| :---------   | :-------------------------------------------------------------------------------------------- |
| `R`          | Radius of the disks, in meters.                                                               |
| `DMIN_KM`    | Minimum distance between two data points in km. Select one of each violating pair at random.  |
| `min_points` | Minimum number of selected points. Reject proposed disks if less selected points within.      |

In [ ]:
R = 80e3
DMIN_KM = 20
min_points = 10

## Package imports:

In [ ]:
import shapely
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj
from pathlib import Path
from zeal2022hf import kindlmann, kindlmann_r
from loaducerf3 import PolygonSelector, Polygon
from pdtoolbox.distributions import *
from pdtoolbox.mle import *
from pdtoolbox.likelihood import *
from pickle import Pickler, Unpickler
from cache import cached_call
from reheatfunq.coverings import random_global_R_disk_coverings
from reheatfunq import GammaConjugatePrior
from reheatfunq.regional.backend import gamma_mle
from cmcrameri.cm import *
from matplotlib.patches import Polygon as MPolygon
from matplotlib.colors import Normalize

Figure style configurations:

In [ ]:
from matplotlib import rcParams
rcParams["text.usetex"] = False
rcParams['font.family'] = 'sans'
rcParams['text.latex.preamble'] = "\\renewcommand{\\familydefault}{\\sfdefault}\n\\usepackage{helvet}"
rcParams['font.size'] = 10

Configure plots to look good on a HiDPI monitor (you may not need the following configuration if you are not using a HiDPI monitor):

In [ ]:
%config InlineBackend.figure_format = 'retina'

## Dataset
Load the data set processed in `01-Load-and-filter-NGHF.ipynb`

In [ ]:
hf_continental = np.load('intermediate/heat-flow-selection-mW_m2.npy')

### Exclude areas used in regional analyses
Load the selectors that will later select the regional aggregate heat flow distributions.
These polygon selectors have been defined in `02-Study-Area-Geometry.ipynb`.

In [ ]:
with open('intermediate/02-Geometry.pickle','rb') as f:
    saf_geometry = Unpickler(f).load()

proj_saf = Proj(saf_geometry["proj_str"])

In [ ]:
mask = np.ones(hf_continental.shape[1], dtype=bool)
hf_xy = np.stack(proj_saf(*hf_continental[1:3,:]), axis=1)

for poly in saf_geometry["selection_polygons_xy"]:
    select = PolygonSelector(Polygon(*poly[:-1].T))
    mask &= ~select.array_mask(hf_xy)
hf_independent = (hf_continental.T)[mask]

### Back-indexing to the NGHF data set
Compute the indices of the masked data within the original NGHF data set (Lucazeau, 2019)
for simple reproducibility. This might or might not be relevant for applications to other
areas and/or data sets.

In [ ]:
hf_continental_indices = np.loadtxt('results/nghf-selection-indices.csv', delimiter=',', dtype=int)

# We also have to update the indices into hf_continental now:
local2continental = np.arange(mask.size)[mask]
local2lucazeau = hf_continental_indices[local2continental]
lucazeau2local = {i : j for i,j in enumerate(local2lucazeau)}

### Create a disk-buffer around the polygons
In the later analysis based on the random distribution of disks all over
Earth surface, we want to prevent the disks from overlapping with the selection
polygons. Using the buffer functionality of Shapely, we can create buffered
polygons (buffer by disk radius) that can prevent by rejecting any disk center
whose center is within the buffer.

In [ ]:
R = 80e3

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal')
buffered_shpoly = shapely.geometry.Polygon()
for poly in saf_geometry["selection_polygons_xy"]:
    ax.plot(*poly.T, color='k')
    shpoly = shapely.geometry.Polygon(poly).buffer(R)
    buffered_shpoly = buffered_shpoly.union(shpoly)

x,y = buffered_shpoly.exterior.coords.xy
ax.plot(x,y,color='gray')
buffered_poly = np.array(x[:-1]), np.array(y[:-1])

In [ ]:
with open('intermediate/03-Buffered-Poly.pickle','wb') as f:
    Pickler(f).dump(buffered_poly)

## Determine Regional Distributions
First off, prepare some code for spatial filtering. So far, we have not checked for double entries to the data base. Also, we might want to remove data points that are too close to each other (circumventing the spatial clustering).

In [ ]:
seed = 498267187

In [ ]:
valid_points, used_data_entries, distributions, distribution_lola, distribution_indices \
    = cached_call(random_global_R_disk_coverings, R, min_points,
                  hf_independent, buffered_poly, saf_geometry["proj_str"], dmin=DMIN_KM*1e3,
                  MAX_DRAW=10000000, N=200, seed=seed)

print("Number of distributions:  ",len(distributions))
print("Maximum distribution size:", max(d.size for d in distributions))

In [ ]:
fig = plt.figure(figsize=(12,4))

# Histogram of sample size:
ax = fig.add_subplot(131)
ax.hist([d.size for d in distributions], bins=20)
ax.set_xlabel('Regional sample size')
ax.set_ylabel('Number of regions')

# Global distribution:
ax = fig.add_subplot(132)
ax.scatter(*hf_independent[:,1:3].T, marker='.', edgecolor='none')
ax.scatter(*np.array(valid_points).T, marker='x')
ax.plot(*proj_saf(x, y, inverse=True), color='gray')

# The study area:
ax = fig.add_subplot(133)
ax.plot(*proj_saf(x, y, inverse=True), color='gray')
ax.set_xlim(ax.get_xlim())
ax.set_ylim(ax.get_ylim())
ax.scatter(*hf_independent[:,1:3].T, marker='.', edgecolor='none')
ax.scatter(*np.array(valid_points).T, marker='x')


Path('figures').mkdir(exist_ok=True)
fig.tight_layout()
fig.savefig('figures/03-RGRDC-sample-sizes-and-locations.pdf')

Mark the data entries that will be used in the later regional analysis:

In [ ]:
j = 0
ude = []
for i,d in enumerate(distributions):
    Ni = d.size
    ude.append([int(local2lucazeau[k])
                for k in used_data_entries[j:j+Ni]])
    j += Ni

used_data_entries = ude

Maximum-likelihood estimates to the regional distributions:

In [ ]:
A, B = np.array([gamma_mle(dist, amin=1.0) for dist in distributions]).T

Export the data:

In [ ]:
Path('results').mkdir(exist_ok=True)

In [ ]:
import json
with open('results/03-gamma-conjugate-prior-results.json','w') as f:
    json.dump([(pts, entries, (a,b))
               for pts, entries, a, b in zip(valid_points, used_data_entries, A, B)],
              f)
pts_feature_list = [{"type" : "Feature",
                     "geometry" : {
                         "type" : "Point",
                         "coordinates" : pt,
                     },
                     "properties" : {
                         "a" : a,
                         "b" : b,
                         "lucazeau2019entries" : entries
                     }} for pt, entries, a, b in zip(valid_points, used_data_entries, A, B)]
geojson_dict = { "type": "FeatureCollection",
                 "name" : "Ziebarth et al. (2021) gamma distribution fits to Lucazeau (2019) heat flow data",
                 "license" : "CC-BY 4.0",
                 "features": pts_feature_list}
with open('results/gamma-conjugate-prior-results.geojson','w') as f:
    json.dump(geojson_dict, f)

## Fit the Gamma Conjugate Prior
We compute the *minimum surprise estimate* (MSE) of the gamma conjugate prior given the distributions.
The MSE minimizes the Kullback-Leibler (KL) distance between conjugate prior $\phi(\alpha,\beta)$ and
the heat flow data.
In particular, for each aggregate heat flow distribution $Q_i = \{q_j\}_i$, a posterior probability
$\pi_i(\alpha, \beta)$ is computed from an "uninformed" prior ($p_i=1$, $s_i=n_i=v_i=0$, see Miller, 1980).
Then, the maximum KL distance $d$ from the conjugate prior to any of the $\pi_i$,
$$
    d = \max\limits_i \left\{ \,\int\limits_{\alpha_\mathrm{min}}^\infty\!\mathrm{d}\alpha
                              \int\limits_{0}^\infty\!\mathrm{d}\beta \,
                              \pi_i(\alpha,\beta)
                              \ln\left(\frac{\pi_i(\alpha,\beta)}{\phi(\alpha,\beta)}\right)     
                      \right\}
$$
is minimized using the SciPy L-BFGS-B optimizer to yield the parameter estimates $\hat{p}$, $\hat{s}$,
$\hat{n}$, $\hat{\nu}$.

In [ ]:
gcp = GammaConjugatePrior.minimum_surprise_estimate(distributions, verbose=True)
gcp

##### Inspect the results

Plot the cost function surrounding the minimum:

In [ ]:
GCP_i = [GammaConjugatePrior(1, 0, 0, 0).updated(dist) for dist in distributions]

def cost_function(p, s, n, v):
    try:
        gcp = GammaConjugatePrior(p, s, n, v)
        return max(gcp.kullback_leibler(gcpi) for gcpi in GCP_i)
    except:
        return np.NaN

In [ ]:
M = 50

In [ ]:
P = np.linspace(np.exp(gcp.lp)-0.01, np.exp(gcp.lp)+0.01, M+1)
S = np.linspace(gcp.s-0.05, gcp.s+0.05, M)
C = np.array([[cost_function(p, s, gcp.n, gcp.v) for p in P] for s in S])

In [ ]:
N = np.linspace(gcp.n-0.01, gcp.n+0.01, M+1)
V = np.linspace(gcp.v-0.01, gcp.v+0.01, M)
C2 = np.array([[cost_function(np.exp(gcp.lp), gcp.s, n, v)
               if (n > v and n-v < 0.008) else np.NaN for n in N] for v in V])

In [ ]:
with plt.rc_context({"font.size" : 10, "xtick.labelsize" : "small", "ytick.labelsize" : "small"}):
    fig = plt.figure(figsize=(6.975, 4.0))
    #ax_bg = fig.add_axes((0,0,1,1))
    cax = fig.add_axes((0.33, 0.11, 0.34, 0.02))
    ax = fig.add_axes((0.09, 0.28, 0.38, 0.71))
    Cmax = 10.0
    mask = C < Cmax
    Cp = C.copy()
    Cp[~mask] = np.NaN
    mask = C2 < Cmax
    Cp2 = C2.copy()
    Cp2[~mask] = np.NaN
    norm = Normalize(min(Cp[mask].min(),Cp2[mask].min()), Cmax)
    h = ax.pcolormesh(P, S, Cp, norm=norm, cmap=batlow, rasterized=True)
    ax.scatter(np.exp(gcp.lp), gcp.s, marker='h', facecolor='w', edgecolor='k', linewidth=0.8,
               label='Optimum')
    ax.set_xlabel('$p$')
    ax.set_ylabel('$s$');
    for tick in ax.get_yticklabels():
        tick.set_rotation(45)
    ax = fig.add_axes((0.59, 0.28, 0.38, 0.71))
    h = ax.pcolormesh(N, V, Cp2, norm=norm, cmap=batlow, rasterized=True)
    ax.scatter(gcp.n, gcp.v, marker='h', facecolor='w', edgecolor='k', linewidth=0.8,
               label='Optimum')
    ax.legend()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.set_xlim((min(xlim[0],ylim[0]), max(xlim[1],ylim[1])))
    ax.set_ylim((min(xlim[0],ylim[0]), max(xlim[1],ylim[1])))
    ax.add_patch(MPolygon([(xlim[0],ylim[0]), (xlim[1],ylim[1]), (xlim[0], ylim[1])], color='lightgray',zorder=0))
    fig.colorbar(h, cax=cax, orientation='horizontal', extend='max')
    cax.set_xlabel('Cost function')
    ax.set_xlabel('$n$')
    ax.set_ylabel('$\\nu$');
    for tick in ax.get_yticklabels():
        tick.set_rotation(45);
    
    fig.savefig('figures/03-Optimization-Result-psnv.pdf', dpi=300)

In [ ]:
Ctmp = C.copy()
Ctmp[np.isnan(Ctmp)] = np.inf
print("final:      ",cost_function(np.exp(gcp.lp), gcp.s, gcp.n, gcp.v))
print("raster best:",Ctmp.min())
imin, jmin = np.unravel_index(np.argmin(Ctmp), Ctmp.shape)

In [ ]:
Ctmp = C2.copy()
Ctmp[np.isnan(Ctmp)] = np.inf
print("final:      ",cost_function(np.exp(gcp.lp), gcp.s, gcp.n, gcp.v))
print("raster best:",Ctmp.min())
kmin, lmin = np.unravel_index(np.argmin(Ctmp), Ctmp.shape)

Plot the result:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
gcp.visualize(ax, distributions, cmap=kindlmann_r,
              q_plot = [(25., 1.0, 50.0, 'lightgray'), (130., 10.0, 1000, 'lightgray')],
              qstd_plot = [2.2, (60., 1.1, 50, 'w')])
fig.tight_layout()
fig.savefig('figures/03-Gamma-Conjugate-Prior.pdf')

Prior predictive:

In [ ]:
QMAX = 250
q = np.linspace(0, QMAX, 500)
pdf = gcp.posterior_predictive(q)
cdf = gcp.posterior_predictive_cdf(q)

In [ ]:
print("Tail >= 250 mWm²:",100 * (1.0 - cdf[-1]),"%")

In [ ]:
z = np.linspace(0, 1, 200)
empirical_cdf_exceedance = np.zeros((z.size, q.size, len(distributions)))
cdf_discrete = np.zeros_like(q)
for i,dist in enumerate(distributions):
    cdf_discrete[:] = 0.0
    for qi in dist:
        cdf_discrete[q >= qi] += 1.0
    cdf_discrete /= dist.size
    for j in range(q.size):
        empirical_cdf_exceedance[z <= cdf_discrete[j], j, i] = 1

empirical_cdf_exceedance = empirical_cdf_exceedance.mean(axis=2)

In [ ]:
fig = plt.figure(figsize=(6.975, 2.6))
ax = fig.add_axes((0.08, 0.175, 0.36, 0.80))
ax.plot(q, 100*pdf, label='Prior predictive\nPDF')
ax.set_ylim(0, 2.55)
ax.set_xlim(0,QMAX)
ax.set_xlabel('Heat flow $q$ ($\mathrm{mWm}^{-2}$)')
ax.set_ylabel('PDF ($10^2\,\\mathrm{mW}^{-1}\\mathrm{m}^2$)')
ax.axvline(68.3, linewidth=0.8, linestyle='--', color='k',
           label='$68.3\,\mathrm{mWm}^{-2}$')
ax.legend(fontsize='small')
ax = fig.add_axes((0.53, 0.175, 0.36, 0.80))
cax = fig.add_axes((0.9, 0.3, 0.02, 0.6))
h = ax.pcolormesh(q, z, empirical_cdf_exceedance, cmap=lisbon, rasterized=True)
ax.plot(q, cdf, color='w', linewidth=3)
ax.plot(q, cdf, color='k', label='Prior predictive\nCDF')
ax.set_ylim(0,1)
ax.set_xlim(0,QMAX)
ax.set_xlabel('Heat flow $q$ ($\mathrm{mWm}^{-2}$)')
ax.set_ylabel('CDF $F$')
fig.colorbar(h, cax=cax)
cax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
cax.set_yticklabels(["0","25","50","75","100"], size='small', rotation=90)
cax.set_ylabel("Fraction of RGRCD empirical\nCDF smaller than $F$ (%)", size='small')
ax.legend(fontsize='small');
fig.savefig('figures/03-Prior-Predictive.pdf', dpi=300)

Export the results:

In [ ]:
np.savetxt('results/05-GCP-Parameters.txt', [[np.exp(gcp.lp), gcp.s, gcp.n, gcp.v]],
           header = "p, s, n, v", delimiter=",")

## References:
> Lucazeau, F. (2019). Analysis and mapping of an updated terrestrial heat
>    flow data set. Geochemistry, Geophysics, Geosystems, 20, 4001– 4024.
>    https://doi.org/10.1029/2019GC008389

### License
```
A notebook to determine the parameters of the gamma conjugate prior
from regional aggregate heat flow distributions.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2019-2022 Deutsches GeoForschungsZentrum Potsdam,
            2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```